# Grattage des conjuguaisons des verbes

Cet algorithme est utilisé pour extraire les conjuguaisons des verbes du site web du **Le conjuguer** du **Le Figaro**
adresse: https://leconjugueur.lefigaro.fr/

In [1]:
import requests
import json
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
chemin_acces_fichier_verbes_sans_info = 'liste_verbes_sans_info.json'
chemin_acces_fichier_verbes_avec_info = 'liste_verbes_avec_info.json'
chemin_acces_fichier_verbes_conjugues_sans_groupe = 'verbes_conjuges_sans_groupe'
chemin_acces_fichier_verbes_conjugues_avec_groupe = 'verbes_conjuges_avec_groupe'


CODE_SUCCES = 200

In [3]:
liste_verbes = []
with open(chemin_acces_fichier_verbes_avec_info) as fichier:
    verbes_par_lettre = json.load(fichier)
    for lettre in verbes_par_lettre:
        liste_verbes.extend(verbes_par_lettre[lettre])
pd.DataFrame(liste_verbes)

,verbe,url,groupe,anglais
0,abaisser,https://leconjugueur.lefigaro.fr/conjugaison/v...,premier groupe,to lower
1,abandonner,https://leconjugueur.lefigaro.fr/conjugaison/v...,premier groupe,to give up
2,abasourdir,https://leconjugueur.lefigaro.fr/conjugaison/v...,deuxième groupe,to stunner
3,abattre,https://leconjugueur.lefigaro.fr/conjugaison/v...,troisième groupe,to shoot down
4,abêtir,https://leconjugueur.lefigaro.fr/conjugaison/v...,deuxième groupe,to abet
...,...,...,...,...
1971,voûter,https://leconjugueur.lefigaro.fr/conjugaison/v...,premier groupe,to vault
1972,vouvoyer,https://leconjugueur.lefigaro.fr/conjugaison/v...,premier groupe,to speak with vous
1973,voyager,https://leconjugueur.lefigaro.fr/conjugaison/v...,premier groupe,to travel
1974,zapper,https://leconjugueur.lefigaro.fr/conjugaison/v...,premier groupe,to zap


In [4]:
pronons_personnels = {
    "j'":'pps', 
    "je":'pps', 
    'tu': 'dps', 
    'il': 'tps',
    'elle': 'tps', 
    'nous': 'ppp',
    'vous': 'dpp',
    'ils': 'tpp',
    'elles': 'tpp'
}

def conjugaison_info_avec_groupe(verbe, groupe, mode, temps, conjugaisons):
    conjugaisons_personnes = []
    if temps == 'impératif':
        personnes = ['tu', 'nous', 'vous']
    else:
        personnes = ['je', 'tu', 'il', 'nous', 'vous', 'ils']
    
    for pers, conjugaison in zip(personnes, conjugaisons):
        conjugaisons_personnes.append(
            {'verbe': verbe, 'mode':mode, 'groupe':groupe, 'personne': pronons_personnels[pers], 'temps': temps, 'conjugaison': conjugaison} 
        )
    return conjugaisons_personnes

def conjugaison_info_sans_groupe(verbe, mode, temps, conjugaisons):
    conjugaisons_personnes = []
    if temps == 'impératif':
        personnes = ['tu', 'nous', 'vous']
    else:
        personnes = ['je', 'tu', 'il', 'nous', 'vous', 'ils']
    
    for pers, conjugaison in zip(personnes, conjugaisons):
        conjugaisons_personnes.append(
            {'verbe': verbe, 'mode':mode, 'temps': temps, 'personne': pronons_personnels[pers], 'conjugaison': conjugaison} 
        )
    return conjugaisons_personnes


In [5]:
'https://leconjugueur.lefigaro.fr/conjugaison/verbe/abasourdir.html'

'https://leconjugueur.lefigaro.fr/conjugaison/verbe/abasourdir.html'

## Partie de grattage web

In [5]:
from tqdm import tqdm
import time
from threading import Thread, RLock
from collections import defaultdict, deque

In [6]:
def grattage(lien:str):
    res = requests.get(lien)
    if res.status_code == CODE_SUCCES:
        return res.status_code, BeautifulSoup(res.text, 'html.parser')
    return res.status_code, None 

### Version iterative (normal)

In [ ]:
verbes_conjuges_sans_groupes = []
iteration = 0
pbar = tqdm(total=len(liste_verbes))
for verbe_info in liste_verbes:
    verbe = verbe_info['verbe']
    conjugaison_url = verbe_info['url']
    code_succes, soup_data = grattage(conjugaison_url)
    if code_succes == CODE_SUCCES:
        contenu = soup_data.find("main").find_all()
        mode = ''
        for element in contenu:
            if (
                element.name == 'h2' 
                and (element.get('class') == ['modeBloc'] or element.get('id') == 'imp')
                ):
                mode = element.text.strip().lower()
                if mode == 'participe':
                    break
            elif (
                element.name == 'div' 
                and mode != ''
                and element.get('class') == ['conjugBloc']
                ):
                try:
                    temps = element.find_all('div')[0].text.strip().lower()
                    # print(mode, '|', temps)
                    verbes_conjuges = ['']
                    sequence = ''
                    for element_desc in element.children:
                        if element_desc.name != 'div':
                            sequence = element_desc.text.strip()
                            if  sequence == '':
                                verbes_conjuges.append('')
                            else:
                                verbes_conjuges[-1] += sequence + ''
                    # verbes_conjugues[-1] += sequence
                    try:
                        verbes_conjuges_sans_groupes.extend(conjugaison_info_sans_groupe(verbe, mode, temps, verbes_conjuges))  
                    except:
                        pass        
                except:
                    pass
    pbar.set_description(f"Iteration {iteration + 1}")
    pbar.update(1)
    iteration +=1
pbar.close()
            

In [ ]:
import pandas as pd
import json
pd.DataFrame(verbes_conjuges_sans_groupes).to_csv(chemin_acces_fichier_verbes_conjugues_sans_groupe+'.csv', index=False)


# Exporter les données vers un fichier JSON
with open(chemin_acces_fichier_verbes_conjugues_sans_groupe+'.json', 'w') as f:
    json.dump(verbes_conjuges_sans_groupes, f, indent=4)


### Version avec programmation parallèle (plus rapide)

Cette version est exponentiellement plus rapide faisant passe le programme d'environ 30min à environ 2min 

1. Creation de packet de verbes de tailles de N avec le dernier packet avec une taille possibllement differente de N

In [7]:
ensembles_verbes = []
ensemble_courant = []
iteration = 0
trame = 50
liste_verbes_queue = deque(liste_verbes)
while liste_verbes_queue:
    if iteration == trame:
        iteration = 0
        ensembles_verbes.append(ensemble_courant)
        ensemble_courant = []
    ensemble_courant.append(liste_verbes_queue.popleft())
    iteration += 1

if len(ensemble_courant) > 0: 
    ensembles_verbes.append(ensemble_courant)
    ensemble_courant = []

2. Definition de la fonction de grattage pour packet de verbes

In [8]:
def grattage_conjugaison_verbes(liste_verbes:list, resultat:list, id_thread=1):
    verbes_conjugaisons = []
    for verbe_info in liste_verbes:
        verbe = verbe_info['verbe']
        groupe = verbe_info['groupe']
        conjugaison_url = verbe_info['url']
        code_succes, soup_data = grattage(conjugaison_url)
        if code_succes == CODE_SUCCES:
            contenu = soup_data.find("main").find_all()
            mode = ''
            for element in contenu:
                if (
                    element.name == 'h2' 
                    and (element.get('class') == ['modeBloc'] or element.get('id') == 'imp')
                    ):
                    mode = element.text.strip().lower()
                    if mode == 'participe':
                        break
                elif (
                    element.name == 'div' 
                    and mode != ''
                    and element.get('class') == ['conjugBloc']
                    ):
                    try:
                        temps = element.find_all('div')[0].text.strip().lower()
                        # print(mode, '|', temps)
                        verbes_conjuges = ['']
                        sequence = ''
                        for element_desc in element.children:
                            if element_desc.name != 'div':
                                sequence = element_desc.text.strip()
                                if  sequence == '':
                                    verbes_conjuges.append('')
                                else:
                                    verbes_conjuges[-1] += sequence + ''
                        try:
                            verbes_conjugaisons.extend(conjugaison_info_avec_groupe(verbe, groupe, mode, temps, verbes_conjuges))  
                        except:
                            pass        
                    except:
                        pass
    with RLock():
        resultat.extend(verbes_conjugaisons)
        print('Thread: ' ,id_thread, 'terminé')

In [9]:
liste_threads = []
verbes_conjuges_avec_groupe = []
nombre_threads = 0
for ensemble_verbes in (ensembles_verbes):
    thread = Thread(target=grattage_conjugaison_verbes, args=(ensemble_verbes, verbes_conjuges_avec_groupe, nombre_threads))
    thread.daemon = True
    thread.start()
    liste_threads.append(thread)
    nombre_threads += 1

while any([thread.is_alive() for thread in liste_threads]):
    time.sleep(1)

Thread:  39 terminé
Thread:  18 terminé
Thread: Thread:  0 terminé
 7 terminé
Thread: Thread:  8 terminé
Thread:  20 terminé
 15 terminé
Thread:  31 terminé
Thread:  1 terminé
Thread:  23 terminé
Thread:  3 terminé
Thread:  37 terminé
Thread:  35 terminé
Thread:  34 terminé
Thread:  16 terminé
Thread:  6 terminé
Thread:  28 terminé
Thread:  19 terminé
Thread: Thread:  14 terminé
 26 terminé
Thread: Thread:  17 terminé
 10 terminé
Thread:  4 terminé
Thread:  33 terminé
Thread:  11 terminé
Thread:  22 terminé
Thread:  21 terminé
Thread:  36 terminé
Thread:  2 terminé
Thread:  25 terminé
Thread:  9 terminé
Thread:  38 terminé
Thread:  5 terminé
Thread:  24 terminé
Thread:  32 terminé
Thread:  12 terminé
Thread:  29 terminé
Thread:  13 terminé
Thread:  30 terminé
Thread:  27 terminé


In [10]:
import pandas as pd
import json
pd.DataFrame(verbes_conjuges_avec_groupe).to_csv(chemin_acces_fichier_verbes_conjugues_avec_groupe+'.csv', index=False)


# Exporter les données vers un fichier JSON
with open(chemin_acces_fichier_verbes_conjugues_avec_groupe+'.json', 'w') as f:
    json.dump(verbes_conjuges_avec_groupe, f, indent=4)


In [11]:
pd.DataFrame(verbes_conjuges_avec_groupe)


,verbe,mode,groupe,personne,temps,conjugaison
0,vêtir,indicatif,troisième groupe,pps,présent,jevêts
1,vêtir,indicatif,troisième groupe,dps,présent,tuvêts
2,vêtir,indicatif,troisième groupe,tps,présent,ilvêt
3,vêtir,indicatif,troisième groupe,ppp,présent,nousvêtons
4,vêtir,indicatif,troisième groupe,dpp,présent,vousvêtez
...,...,...,...,...,...,...
187963,planter,impératif,premier groupe,dps,présent,plantons
187964,planter,impératif,premier groupe,tps,présent,plantez
187965,planter,impératif,premier groupe,pps,passé,aie planté
187966,planter,impératif,premier groupe,dps,passé,ayons planté
